In [53]:
import collections
import pandas as pd
import numpy as np
import tensorflow as tf
import tensorflow_federated as tff
import pickle


np.random.seed(0)

tff.federated_computation(lambda: 'Hello, World!')()

b'Hello, World!'

In [29]:
daily_df = pd.read_csv("/content/anonym_aamos00_dailyquestionnaire.csv")
endquest_df = pd.read_csv("/content/anonym_aamos00_endquestionnaire.csv")
environment_df = pd.read_csv("/content/anonym_aamos00_environment.csv")
patient_info_df = pd.read_csv("/content/anonym_aamos00_patient_info.csv")
peakflow_df = pd.read_csv("/content/anonym_aamos00_peakflow.csv")
inhaler_df = pd.read_csv("/content/anonym_aamos00_smartinhaler.csv")
watch_df1 = pd.read_csv("/content/anonym_aamos00_smartwatch1.csv")
watch_df2 = pd.read_csv("/content/anonym_aamos00_smartwatch2.csv")
watch_df3 = pd.read_csv("/content/anonym_aamos00_smartwatch3.csv")
weekly_df = pd.read_csv("/content/anonym_aamos00_weeklyquestionnaire.csv")
watch_activity_lookup = pd.read_excel("/content/aamos00_data_dictionary.xlsx", sheet_name="miband_activity_lookup")


In [30]:
daily_df.head()

,user_key,date,daily_night_symp,daily_day_symp,daily_limit_activity,daily_prev_inhaler,daily_relief_inhaler,daily_triggers
0,113,0,True,True,False,2,5,"5,9"
1,113,1,False,False,False,2,3,"5,9"
2,113,2,False,True,False,2,5,"5,9"
3,113,3,False,False,False,2,1,"5,12"
4,113,4,False,True,False,2,3,"5,9"


In [31]:
endquest_df.head()

,user_key,completion_date,i_think_that_i_would_like_to_use_this_system_frequently,i_found_the_system_unnecessarily_complex,i_thought_the_system_was_easy_to_use,i_think_that_i_would_need_the_support_of_a_technical_person_to_be_able_to_use_this_system,i_found_the_various_functions_in_this_system_were_well_integrated,i_thought_there_was_too_much_inconsistency_in_this_system,i_would_imagine_that_most_people_would_learn_to_use_this_system_very_quickly,i_found_the_system_very_cumbersome_to_use,...,awareness_a_this_study_has_increased_my_awareness_of_the_importance_of_addressing_the_health_behaviour,knowledge_a_this_study_has_increased_my_knowledge_understanding_of_the_health_behaviour,attitudes_a_the_study_has_changed_my_attitudes_toward_improving_this_health_behaviour,intention_to_change_a_the_study_has_increased_my_intentions_motivation_to_address_this_health_behaviour,help_seeking_a_this_app_would_encourage_me_to_seek_further_help_to_address_the_health_behaviour_if_i_needed_it,behaviour_change_a_use_of_this_app_will_increase_decrease_the_health_behaviour,i_would_pay_for_the_find_air_one_rrp_59_00_eur_per_year,i_would_pay_for_the_smart_asthma_smart_peak_flow_meter_rrp_59_60_gbp,i_would_pay_for_the_mi_band_3_smartwatch_rrp_25_00_gbp,i_had_received_training_in_taking_peak_flow_measurements_with_a_peak_flow_meter_from_a_healthcare_professional_e_g_asthma_doctor_nurse_pharmacist_etc_prior_to_joining_this_study
0,113,178,(4) Agree,(3) Neither agree nor disagree,(5) Strongly agree,(1) Strongly disagree,(5) Strongly agree,(2) Disagree,(5) Strongly agree,(2) Disagree,...,(5) Strongly agree,(3) Neither agree nor disagree,(4) Agree,(4) Agree,(5) Strongly agree,(4) Agree,(3) Neither agree nor disagree,(1) Strongly disagree,(2) Disagree,Yes
1,190,166,(4) Agree,(2) Disagree,(5) Strongly agree,(2) Disagree,(4) Agree,(2) Disagree,(4) Agree,(2) Disagree,...,(4) Agree,(4) Agree,(4) Agree,(4) Agree,(4) Agree,(4) Agree,(2) Disagree,(4) Agree,(4) Agree,Yes
2,294,179,(4) Agree,(2) Disagree,(4) Agree,(1) Strongly disagree,(4) Agree,(2) Disagree,(5) Strongly agree,(2) Disagree,...,(5) Strongly agree,(5) Strongly agree,(5) Strongly agree,(4) Agree,(5) Strongly agree,(5) Strongly agree,(2) Disagree,(2) Disagree,(4) Agree,Yes
3,343,176,(2) Disagree,(3) Neither agree nor disagree,(2) Disagree,(4) Agree,(3) Neither agree nor disagree,(2) Disagree,(4) Agree,(3) Neither agree nor disagree,...,(3) Neither agree nor disagree,(2) Disagree,(2) Disagree,(2) Disagree,(3) Neither agree nor disagree,(2) Disagree,(1) Strongly disagree,(1) Strongly disagree,(1) Strongly disagree,Yes
4,447,143,(2) Disagree,(3) Neither agree nor disagree,(2) Disagree,(2) Disagree,(3) Neither agree nor disagree,(5) Strongly agree,(2) Disagree,(2) Disagree,...,(4) Agree,(2) Disagree,(2) Disagree,(2) Disagree,(4) Agree,(2) Disagree,(2) Disagree,(2) Disagree,(1) Strongly disagree,Yes


In [32]:
environment_df.head()

,user_key,date,temperature,temperature_min,temperature_max,pressure,humidity,wind_speed,wind_deg,aqi,...,no,no2,o3,so2,pm2_5,pm10,nh3,grass_pollen,tree_pollen,weed_pollen
0,113,1,15.91,14.21,17.20,1015,88,0.89,53,1.0,...,0.19,8.65,47.92,2.89,7.55,8.51,0.82,High,Low,Moderate
1,113,2,16.67,14.92,18.71,1014,96,0.45,24,1.0,...,0.55,15.94,28.97,6.68,3.97,4.82,0.29,High,Moderate,High
2,113,3,17.07,15.00,18.31,1015,79,1.34,17,1.0,...,0.49,8.14,56.51,3.04,3.93,7.39,0.65,High,Moderate,High
3,113,4,18.08,15.22,20.13,1017,78,0.89,16,1.0,...,0.19,2.29,79.39,0.78,2.91,4.09,1.20,High,Moderate,Moderate
4,113,5,20.82,17.81,22.86,1015,86,1.79,135,1.0,...,0.11,4.84,78.68,1.85,8.35,9.05,1.63,High,Low,Moderate


In [33]:
patient_info_df.head()

,user_key,sex,age_range,bmi_range,age_diagnosed_range,max_pef_expected,smoker,pack_years,race,severity,...,position_start_date,pef_start_date,inhaler_start_date,daily_end_date,weekly_end_date,miband_end_date,position_end_date,pef_end_date,inhaler_end_date,phase2_end
0,113,female,30-39yo,Obesity,7-11yo,485,Never (<100 cigarettes in lifetime),0.00,White,Severe,...,1,1.0,0.0,183,183,183.0,183,40.0,177.0,183
1,190,male,50+yo,Pre-obesity,19+yo,575,Previous,18.75,White,Severe,...,1,6.0,1.0,183,182,148.0,183,183.0,19.0,183
2,217,female,30-39yo,Obesity,19+yo,495,Previous,6.30,White,Moderate,...,0,0.0,1.0,1,0,1.0,1,0.0,1.0,1
3,278,female,18-29yo,Pre-obesity,12-18yo,495,Previous,1.20,White,Moderate,...,0,NaN,-20.0,15,15,15.0,13,NaN,106.0,106
4,294,female,50+yo,Obesity,0-6yo,455,Never (<100 cigarettes in lifetime),0.00,White,Moderate,...,0,0.0,-5.0,183,182,183.0,183,183.0,184.0,184


In [34]:
peakflow_df.head()

,user_key,date,hour,pef_max,pefs,morning
0,113,1,7,469,469|99|144,True
1,113,1,15,310,310|174|175,False
2,113,3,13,290,249|290|207,True
3,113,40,16,557,71|557|525,False
4,190,6,7,333,333|307|150,True


In [35]:
inhaler_df.head()

,user_key,date,hour,name
0,113,0,15,VENTOLIN
1,113,0,15,VENTOLIN
2,113,0,15,VENTOLIN
3,113,0,15,VENTOLIN
4,113,0,15,VENTOLIN


In [36]:
watch_df1.head()

,user_key,date,time,activity_type,intensity,steps,hr
0,113,0,09:35:00,80,31,0,NaN
1,113,0,09:36:00,96,6,0,72.0
2,113,0,09:37:00,80,0,0,73.0
3,113,0,09:38:00,80,13,0,74.0
4,113,0,09:39:00,90,20,0,75.0


In [37]:
watch_df2.head()

,user_key,date,time,activity_type,intensity,steps,hr
0,447,70,06:24:00,80,70,0,91.0
1,447,70,06:25:00,80,56,0,91.0
2,447,70,06:26:00,17,72,46,90.0
3,447,70,06:27:00,16,60,0,90.0
4,447,70,06:28:00,96,66,0,89.0


In [38]:
watch_df3.head()

,user_key,date,time,activity_type,intensity,steps,hr
0,939,99,13:21:00,80,17,0,94.0
1,939,99,13:22:00,80,20,0,97.0
2,939,99,13:23:00,80,3,0,96.0
3,939,99,13:24:00,80,48,0,95.0
4,939,99,13:25:00,17,58,38,95.0


In [39]:
weekly_df.head()

,user_key,date,weekly_night_symp,weekly_day_symp,weekly_limit_activity,weekly_short_breath,weekly_wheeze,weekly_relief_inhaler,weekly_doc,weekly_hospital,weekly_er,weekly_oral
0,113,1,2.0,3.0,1.0,2,2,3,-4,0,0,2.0
1,113,7,2.0,3.0,2.0,2,2,2,-5,0,0,2.0
2,113,14,5.0,5.0,5.0,4,4,5,"-1,-3",0,-2,3.0
3,113,22,4.0,5.0,4.0,4,2,4,-3,0,0,3.0
4,113,28,3.0,3.0,3.0,3,2,3,0,0,0,3.0


In [40]:
watch_activity_lookup.head()

,number,activity
0,1,walk
1,16,walk
2,33,walk
3,3,not_worn
4,6,charging


In [41]:

# Data preprocessing
# Smart peak flow meter data processing
peakflow_day_df = peakflow_df.groupby(['user_key', 'date']).agg(pef_max=('pef_max', 'max')).reset_index()
peakflow_day_df['pef_norm'] = peakflow_day_df['pef_max'] / peakflow_day_df['pef_max'].max()


In [42]:

# Smart inhaler data processing
inhaler_day_df = inhaler_df.groupby(['user_key', 'date']).size().reset_index(name='relief_use')


In [43]:

# Smartwatch data processing
watch_df = pd.concat([watch_df1, watch_df2, watch_df3])
watch_day_df = watch_df.merge(watch_activity_lookup, left_on='activity_type', right_on='number', how='left')
watch_day_df = watch_day_df.groupby(['user_key', 'date']).agg(steps=('steps', 'sum'),
                                                             mean_hr=('hr', 'mean'),
                                                             sleep_minutes=('activity', lambda x: (x == 'sleep').sum())).reset_index()
watch_day_df = watch_day_df[(~watch_day_df['mean_hr'].isna()) & (watch_day_df['mean_hr'] > 1)]


In [44]:
# Weekly questionnaire data processing
adverse_events = weekly_df[['user_key', 'date', 'weekly_doc', 'weekly_hospital', 'weekly_er', 'weekly_oral']]
adverse_events = adverse_events[(adverse_events['weekly_doc'] != "0") | (adverse_events['weekly_hospital'] != "0") |
                                (adverse_events['weekly_er'] != "0") | (adverse_events['weekly_oral'] == "3")]


In [45]:
# Converting answers to dates
adverse_events[['weekly_doc', 'weekly_hospital', 'weekly_er']] = adverse_events[
    ['weekly_doc', 'weekly_hospital', 'weekly_er']].applymap(lambda x: int(float(str(x).split(',')[0])) if pd.notnull(x) else np.nan)
adverse_events_long = adverse_events.melt(id_vars=['user_key', 'date'], var_name='event_type', value_name='days')

# Handling NaN values in the 'days' column
adverse_events_long['days'] = adverse_events_long['days'].fillna(0)

# Multiply days by 'D' frequency to convert to Timedelta objects
adverse_events_long['timedelta_days'] = adverse_events_long['days'].astype(int) * pd.Timedelta('1D')
# Add timedelta days to date column where event_type is not "weekly_oral"
adverse_events_long['date'] = np.where(adverse_events_long['event_type'] != "weekly_oral",
                                       adverse_events_long['date'] + adverse_events_long['timedelta_days'].astype('timedelta64[D]'),
                                       adverse_events_long['date'])


In [46]:
  # Cleaning up
adverse_events_day_df = adverse_events_long[adverse_events_long['days'] != 0].drop(columns='days')
adverse_events_day_df = adverse_events_day_df.pivot_table(index=['user_key', 'date'], columns='event_type',
                                                          aggfunc='size', fill_value=0).reset_index()



In [47]:
# Joining all data
all_data_day_df = daily_df.merge(environment_df, on=['user_key', 'date'], how='outer')
all_data_day_df = all_data_day_df.merge(peakflow_day_df, on=['user_key', 'date'], how='outer')
all_data_day_df = all_data_day_df.merge(inhaler_day_df, on=['user_key', 'date'], how='outer')
all_data_day_df = all_data_day_df.merge(watch_day_df, on=['user_key', 'date'], how='outer')
all_data_day_df = all_data_day_df.merge(adverse_events_day_df, on=['user_key', 'date'], how='outer')


In [48]:
len(all_data_day_df)

2054

In [49]:
all_data_day_df.head()

,user_key,date,daily_night_symp,daily_day_symp,daily_limit_activity,daily_prev_inhaler,daily_relief_inhaler,daily_triggers,temperature,temperature_min,...,pef_max,pef_norm,relief_use,steps,mean_hr,sleep_minutes,weekly_doc,weekly_er,weekly_hospital,weekly_oral
0,113,0.0,True,True,False,2.0,5.0,"5,9",NaN,NaN,...,NaN,NaN,8.0,3035.0,76.610656,400.0,NaN,NaN,NaN,NaN
1,113,1.0,False,False,False,2.0,3.0,"5,9",15.91,14.21,...,469.0,0.733959,13.0,19591.0,83.728114,582.0,0.0,0.0,0.0,1.0
2,113,2.0,False,True,False,2.0,5.0,"5,9",16.67,14.92,...,NaN,NaN,10.0,17615.0,85.743478,475.0,1.0,0.0,0.0,0.0
3,113,3.0,False,False,False,2.0,1.0,"5,12",17.07,15.00,...,290.0,0.453834,NaN,20264.0,89.924469,507.0,NaN,NaN,NaN,NaN
4,113,4.0,False,True,False,2.0,3.0,"5,9",18.08,15.22,...,NaN,NaN,3.0,9266.0,84.513693,535.0,NaN,NaN,NaN,NaN


In [50]:
# Handling missing values
all_data_day_df['relief_use'] = all_data_day_df['relief_use'].fillna(0)
all_data_day_df[['weekly_doc', 'weekly_hospital', 'weekly_er', 'weekly_oral']] = all_data_day_df[
    ['weekly_doc', 'weekly_hospital', 'weekly_er', 'weekly_oral']].fillna(False)


In [51]:
all_data_day_df.head()

,user_key,date,daily_night_symp,daily_day_symp,daily_limit_activity,daily_prev_inhaler,daily_relief_inhaler,daily_triggers,temperature,temperature_min,...,pef_max,pef_norm,relief_use,steps,mean_hr,sleep_minutes,weekly_doc,weekly_er,weekly_hospital,weekly_oral
0,113,0.0,True,True,False,2.0,5.0,"5,9",NaN,NaN,...,NaN,NaN,8.0,3035.0,76.610656,400.0,False,False,False,False
1,113,1.0,False,False,False,2.0,3.0,"5,9",15.91,14.21,...,469.0,0.733959,13.0,19591.0,83.728114,582.0,0.0,0.0,0.0,1.0
2,113,2.0,False,True,False,2.0,5.0,"5,9",16.67,14.92,...,NaN,NaN,10.0,17615.0,85.743478,475.0,1.0,0.0,0.0,0.0
3,113,3.0,False,False,False,2.0,1.0,"5,12",17.07,15.00,...,290.0,0.453834,0.0,20264.0,89.924469,507.0,False,False,False,False
4,113,4.0,False,True,False,2.0,3.0,"5,9",18.08,15.22,...,NaN,NaN,3.0,9266.0,84.513693,535.0,False,False,False,False


In [52]:

df = pd.DataFrame(all_data_day_df)
all_data = df.fillna(False, inplace=True)
data = df.replace({True: 1, False: 0})
data.head()


,user_key,date,daily_night_symp,daily_day_symp,daily_limit_activity,daily_prev_inhaler,daily_relief_inhaler,daily_triggers,temperature,temperature_min,...,pef_max,pef_norm,relief_use,steps,mean_hr,sleep_minutes,weekly_doc,weekly_er,weekly_hospital,weekly_oral
0,113,0.0,1,1,0,2.0,5.0,"5,9",0.00,0.00,...,0.0,0.000000,8.0,3035.0,76.610656,400.0,0,0,0,0
1,113,1.0,0,0,0,2.0,3.0,"5,9",15.91,14.21,...,469.0,0.733959,13.0,19591.0,83.728114,582.0,0,0,0,1
2,113,2.0,0,1,0,2.0,5.0,"5,9",16.67,14.92,...,0.0,0.000000,10.0,17615.0,85.743478,475.0,1,0,0,0
3,113,3.0,0,0,0,2.0,1.0,"5,12",17.07,15.00,...,290.0,0.453834,0.0,20264.0,89.924469,507.0,0,0,0,0
4,113,4.0,0,1,0,2.0,3.0,"5,9",18.08,15.22,...,0.0,0.000000,3.0,9266.0,84.513693,535.0,0,0,0,0


In [56]:
from google.colab import drive
import pickle

drive.mount('/content/drive')
file_path = '/content/drive/My Drive/preprocessed_data.pkl'

with open(file_path, 'wb') as f:
    pickle.dump(data, f)

print("Preprocessed data saved successfully to:", file_path)

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Preprocessed data saved successfully to: /content/drive/My Drive/preprocessed_data.pkl
